In [1]:
# This notebook imports CRSP data, combines and cleans it, creating full data set
# also creates a combined stock price plus treating treated event flag data set

In [2]:
# Combine and Clean CRSP Data
import cPickle as pickle
from datetime import datetime
import funcy
import itertools
import numpy as np
import pandas as pd
import plotly
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import pytz
import os

from analysis.utilities.data_utils import clean_columns, strip_upcase
from analysis.utilities.dropbox_client import DropboxAPI

DROPBOX_ANALYSIS_DATA = '/data/intermediate'
DROPBOX_DATA = '/data'
DATA = os.path.join(os.path.abspath(os.curdir), 'analysis/Data')
TAB_CRSP_FILES = ['output1-50.txt', 'output51-100.txt', 'output101-150.txt']
CSV_CRSP_FILES = ['output_200+.csv']
TT_FNAME = 'treating_treated_ticker'
FULL_CSP_FNAME = 'full_CRSP'
CRSP_LONG_FNAME = 'PRC_RET_VOL_long'
CRSP_FLAG_FNAME = 'crsp_event_flags'
# This compiles all current CRSP data, reads in treating_treated data set and does some summary data viz 

In [3]:
TTYPES = ['TREATING', 'TREATED']
TREATING = 'TREATING'
TREATED = 'TREATED'
TREATING_DT = 'TREATING_DISCONTINUED_IKT_DATE'
TREATED_DT = 'DISCONTINUED_IKT_DATE'
FLAGS = ['SAMEMKT_SAMETECH', 'SAMEMKT_DIFFTECH', 'DIFFMKT_SAMETECH']

In [4]:
PIVOTS = ['RET', 'VOL', 'PRC']

In [5]:
UPLOAD_DATA = True

In [6]:
d = DropboxAPI()


In [7]:
crsp_csv = [d.search_and_download(f, DROPBOX_DATA, DATA, ',') for f in CSV_CRSP_FILES]
crsp_tab = [d.search_and_download(f, DROPBOX_DATA, DATA, '\t') for f in TAB_CRSP_FILES]

Searching output_200+.csv in path /data
Results found in dropbox, downloading /data/crsp/output_200+.csv
downloading /data/crsp/output_200+.csv


/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning:

Columns (9,48,56) have mixed types. Specify dtype option on import or set low_memory=False.



Searching output1-50.txt in path /data
Results found in dropbox, downloading /data/crsp/output1-50.txt
downloading /data/crsp/output1-50.txt
Searching output51-100.txt in path /data
Results found in dropbox, downloading /data/crsp/output51-100.txt
downloading /data/crsp/output51-100.txt
Searching output101-150.txt in path /data
Results found in dropbox, downloading /data/crsp/output101-150.txt
downloading /data/crsp/output101-150.txt


In [8]:
# Concatenate all CRSP files
ds = pd.concat(crsp_tab, axis=0)
ds2 = pd.concat(crsp_csv, axis=0)

In [9]:
ds.columns = map(str.upper, ds.columns)
ds2.columns = map(str.upper, ds2.columns)
print ds2.columns


Index([u'PERMNO', u'DATE', u'NAMEENDT', u'SHRCD', u'EXCHCD', u'SICCD',
       u'NCUSIP', u'TICKER', u'COMNAM', u'SHRCLS', u'TSYMBOL', u'NAICS',
       u'PRIMEXCH', u'TRDSTAT', u'SECSTAT', u'PERMCO', u'ISSUNO', u'HEXCD',
       u'HSICCD', u'CUSIP', u'DCLRDT', u'DLAMT', u'DLPDT', u'DLSTCD',
       u'NEXTDT', u'PAYDT', u'RCRDDT', u'SHRFLG', u'HSICMG', u'HSICIG',
       u'DISTCD', u'DIVAMT', u'FACPR', u'FACSHR', u'ACPERM', u'ACCOMP',
       u'NWPERM', u'DLRETX', u'DLPRC', u'DLRET', u'TRTSCD', u'NMSIND',
       u'MMCNT', u'NSDINX', u'BIDLO', u'ASKHI', u'PRC', u'VOL', u'RET', u'BID',
       u'ASK', u'SHROUT', u'CFACPR', u'CFACSHR', u'OPENPRC', u'NUMTRD',
       u'RETX', u'VWRETD', u'VWRETX', u'EWRETD', u'EWRETX', u'SPRTRN'],
      dtype='object')


In [10]:
# change date column to a date objct
# Note that the two types of files have different date formats
# This process generally takes about 2 minutes
ds2['DATE_OBJ']= ds2['DATE'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))
ds['DATE_OBJ']= ds['DATE'].apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y'))

In [11]:
# concat the two data sets together
ds_merged = ds2.append(ds, ignore_index=True)

In [12]:
# drop date column because its now meaningless
ds_merged.drop('DATE', axis=1, inplace=True)

In [13]:
# check for duplicate data downloads in data set
ds_merged.sort_values(by=['TICKER', 'DATE_OBJ'], inplace=True)

# drop return column and join with serires of coerced numeric values
ds_merged2 = ds_merged.drop(['PERMNO', 'PERMCO', 'PRC', 'VOL', 'RET', 'NUMTRD', 'RETX', 'SPRTRN'], axis=1).join(ds_merged[
        ['PERMNO', 'PERMCO', 'PRC', 'VOL', 'RET', 'NUMTRD', 'RETX', 'SPRTRN']].apply(pd.to_numeric, errors='coerce'))

In [14]:
# Check for duplicate dates
ds_merged2.loc[ds_merged.duplicated(['TICKER', 'DATE_OBJ'], keep=False), ['TICKER', 'DATE_OBJ', 'PRC']]

,TICKER,DATE_OBJ,PRC
367006,ACL,2011-04-11,NaN
367007,ACL,2011-04-11,NaN
354597,ADLR,2011-12-12,NaN
354598,ADLR,2011-12-12,NaN
159676,AGN,2015-03-17,NaN
159677,AGN,2015-03-17,NaN
378660,ANIP,2013-07-18,7.1490
610984,ANIP,2013-07-18,7.1490
378661,ANIP,2013-07-19,7.0000
610985,ANIP,2013-07-19,7.0000


In [15]:
# drop duplicates possibly from duplicate downloads of same ticker, Nans should show up first 
# sorted 
ds_short = ds_merged2.drop_duplicates(subset=['TICKER', 'DATE_OBJ'], keep='last')

In [16]:
# When CRSP doesn't have a final closing price, they will use the average of the bid/ask spread and denote this with a - sign
# we don't want neg stock values, so the price column is reassigned to the abs()
ds_short.rename(columns={'PRC': 'PRC_NEGATIVE'}, inplace=True)
ds_short.loc[:, 'PRC'] = ds_short['PRC_NEGATIVE'].abs()

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [17]:
del ds, ds2, ds_merged, ds_merged2

In [18]:
# pickle result and upload to dropbox
if UPLOAD_DATA:
    d.pickle_upload_dataset(ds_short, '{}.p'.format(FULL_CSP_FNAME), DATA, DROPBOX_DATA)

Serializing to local path /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/full_CRSP.p
Uploading /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/full_CRSP.p to Dropbox as /data/full_CRSP.p


In [19]:
# Create pivot-ed data set versions for both price and returns

pivot_fnc = lambda x: pd.pivot_table(ds_short, values=x, index='DATE_OBJ', columns='TICKER')
frames = []
for p in PIVOTS:
    # create pivot_dataframe
    x = None
    x = pivot_fnc(p)
    x.columns = map(lambda x: '{0}_{1}'.format(x, p), x.columns)
    frames.append(x)

In [20]:
m1 = pd.merge(frames[0], frames[1], left_index=True, right_index=True, how='outer')
m2 =  pd.merge(m1, frames[2], left_index=True, right_index=True, how='outer')

In [21]:
del m1, frames

In [22]:
# sort columns
m2 = m2[sorted(m2.columns)]

In [24]:
# Save dataset
if UPLOAD_DATA:
    d.pickle_upload_dataset(m2, '{}.p'.format(CRSP_LONG_FNAME), DATA, DROPBOX_ANALYSIS_DATA)

Serializing to local path /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/PRC_RET_VOL_long.p
Uploading /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/PRC_RET_VOL_long.p to Dropbox as /data/intermediate/PRC_RET_VOL_long.p


In [25]:
m2.head()

,ABBV_PRC,ABBV_RET,ABBV_VOL,ABEO_PRC,ABEO_RET,ABEO_VOL,ABIO_PRC,ABIO_RET,ABIO_VOL,ABT_PRC,...,ZFGN_VOL,ZGNX_PRC,ZGNX_RET,ZGNX_VOL,ZIOP_PRC,ZIOP_RET,ZIOP_VOL,ZSAN_PRC,ZSAN_RET,ZSAN_VOL
DATE_OBJ,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Read in Treating/Treated Data Set
# r = d.search('{}.csv'.format(TT_FNAME), False)
tt_ds = d.search_and_download('{}'.format(TT_FNAME), DROPBOX_ANALYSIS_DATA, DATA, ',', False)

#  date columns to date without time resolution
tt_ds[TREATED_DT]= tt_ds[TREATED_DT].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
tt_ds[TREATING_DT]= tt_ds[TREATING_DT].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

Searching local path
Full path /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/treating_treated_ticker.csv


In [27]:
# Pickle Data Set and Save to Dropbox
if UPLOAD_DATA:
    d.pickle_upload_dataset(tt_ds, '{}.p'.format(TT_FNAME), DATA, DROPBOX_ANALYSIS_DATA)

Serializing to local path /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/treating_treated_ticker.p
Uploading /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/treating_treated_ticker.p to Dropbox as /data/intermediate/treating_treated_ticker.p


In [28]:
def get_ticker_events(ticker, flag, ttype, event_df, date_cols=[TREATED_DT, TREATING_DT]):
    # get dates where ticker flagged for 'flag_name' event
    # get instances where treating/treated ticker symbol == to ticker type
    # ticker is ticker symbol, flag is string of 'Treating_
    parent_ts = '{}_PARENT_TICKER_SYMBOL'.format(ttype)
    ts = '{}_TICKER_SYMBOL'.format(ttype)
    events = event_df[(event_df[parent_ts].isin([ticker])) | (event_df[ts].isin([ticker]))]
    
    # find dates where each specific flag has value for this ticker, if none, continue to next flag
    if len(events) == 0:
        return None
    event_dates = events.loc[events[flag] > 0, funcy.concat(date_cols, [flag])]
    return event_dates

In [29]:
# Pickle and read in treating treated data set
# create long version of data set
# for each ticker, create columns with flag for 6 different options
# flag_cols = ['{0}_{1}'.format(t, f) for (t, f) in itertools.product(TTYPES, FLAGS)],
def create_event_flag_series(ticker, flag, ttype, event_df, dt_var=TREATING_DT):
    series_name = '{0}_{1}_{2}'.format(ticker, ttype, flag)
    ticker_series = None
    events_df = get_ticker_events(ticker, flag, ttype, event_df)
    if events_df is None or events_df.empty:
        print('{} has no events'.format(series_name))
        return None
    event_dates = events_df[dt_var].unique()
    print('{} has {} events'.format(series_name, len(events_df)))
    # create series using only unique dates (multiple events may happen on same day)
    ticker_series = pd.Series(1, index=pd.DatetimeIndex(event_dates))
    ticker_series.rename(series_name, inplace=True)
    return ticker_series



In [30]:
tt_ds.loc[:, [TREATED_DT, TREATING_DT]]


,DISCONTINUED_IKT_DATE,TREATING_DISCONTINUED_IKT_DATE
0,NaT,2000-08-03
1,NaT,2000-08-03
2,2001-10-16,2000-08-03
3,2001-11-16,2000-08-03
4,2003-05-12,2000-08-03
5,2001-10-04,2000-08-03
6,NaT,2000-08-03
7,NaT,2000-08-03
8,2001-11-16,2000-08-03
9,2003-05-12,2000-08-03


In [31]:
def create_own_drug_disc_dates(ticker, event_df):
    # for each ticker, get unique dates when either it is a treating event OR 
    # a drug is discontinued when it is in the treated group
    treating_parent_ts = '{}_PARENT_TICKER_SYMBOL'.format(TREATING)
    treating_ts = '{}_TICKER_SYMBOL'.format(TREATING)
    treated_parent_ts = '{}_PARENT_TICKER_SYMBOL'.format(TREATED)
    treated_ts = '{}_TICKER_SYMBOL'.format(TREATED)
    # get dates of when in treated group and discontinues drug
    # when compnay is in the treated group and has a drug disc date in treated datetime
    own_fails = event_df.loc[
        (event_df[treated_parent_ts] == ticker) | (event_df[treated_ts] == ticker), TREATED_DT].dropna().unique()
    treating_fails = event_df.loc[
        (event_df[treating_parent_ts] == ticker) | (event_df[treating_ts] == ticker), TREATING_DT].dropna().unique()
    fails = pd.Series(1, index=pd.DatetimeIndex(np.concatenate([own_fails, treating_fails], axis=0)))
    # remove duplicates
    grouped = fails.groupby(level=0)
    fails2 = grouped.last()
    fails_name = '{0}_DISC'.format(ticker)
    fails2.rename(fails_name, inplace=True)
    return fails2.sort_index()

In [32]:
UNIQUE_TICKERS = ds_short.TICKER.dropna().unique()

flag_series = [
    create_event_flag_series(ticker, flag, ttype, tt_ds)
        for ticker in UNIQUE_TICKERS for ttype in TTYPES for flag in FLAGS]

AB_TREATING_SAMEMKT_SAMETECH has no events
AB_TREATING_SAMEMKT_DIFFTECH has no events
AB_TREATING_DIFFMKT_SAMETECH has no events
AB_TREATED_SAMEMKT_SAMETECH has no events
AB_TREATED_SAMEMKT_DIFFTECH has 486 events
AB_TREATED_DIFFMKT_SAMETECH has 76 events
ABBV_TREATING_SAMEMKT_SAMETECH has 12 events
ABBV_TREATING_SAMEMKT_DIFFTECH has 2141 events
ABBV_TREATING_DIFFMKT_SAMETECH has 55 events
ABBV_TREATED_SAMEMKT_SAMETECH has 2 events
ABBV_TREATED_SAMEMKT_DIFFTECH has 437 events
ABBV_TREATED_DIFFMKT_SAMETECH has 31 events
ABEO_TREATING_SAMEMKT_SAMETECH has no events
ABEO_TREATING_SAMEMKT_DIFFTECH has no events
ABEO_TREATING_DIFFMKT_SAMETECH has no events
ABEO_TREATED_SAMEMKT_SAMETECH has 1 events
ABEO_TREATED_SAMEMKT_DIFFTECH has 307 events
ABEO_TREATED_DIFFMKT_SAMETECH has 14 events
ABIO_TREATING_SAMEMKT_SAMETECH has no events
ABIO_TREATING_SAMEMKT_DIFFTECH has 146 events
ABIO_TREATING_DIFFMKT_SAMETECH has no events
ABIO_TREATED_SAMEMKT_SAMETECH has no events
ABIO_TREATED_SAMEMKT_DIFFTEC

In [33]:
fail_series = [
    create_own_drug_disc_dates(ticker, tt_ds) for ticker in UNIQUE_TICKERS]

In [34]:
flag_df = pd.concat(flag_series, axis=1).sort_index()

/Users/lrraymond13/.virtualenvs/ds3/lib/python2.7/site-packages/pandas/indexes/base.py:1746: FutureWarning:

In the future, 'NAT < x' and 'x < NAT' will always be False.



In [35]:
fail_df = pd.concat(fail_series, axis=1).sort_index()

In [36]:
flag_df2 = pd.concat([flag_df, fail_df], axis=1).sort_index()

In [37]:
# clear memory
del flag_df, fail_df

In [38]:
# check if there are any CRSP tickers with no events and remove them
flag_tickers = set(map(lambda x: x.split('_')[0], flag_df2.columns))
extra_tickers = [x for x in UNIQUE_TICKERS if x not in flag_tickers]
print extra_tickers

[]


In [39]:
cols_to_drop = ['{0}_{1}'.format(t, p) for t in extra_tickers for p in PIVOTS]
# print cols_to_drop
m3 = m2.drop(labels=cols_to_drop, axis=1)

In [40]:
del flag_series, m2

In [41]:
TREATING_DT in m3.columns

False

In [42]:
# now, merge the flag data set with the CRSP dataset
crsp_flag_dataset = pd.concat([flag_df2, m3], axis=1).sort_index()

In [43]:
# sort columns
crsp_flag_dataset = crsp_flag_dataset[sorted(crsp_flag_dataset.columns)]

In [44]:
d.pickle_upload_dataset(crsp_flag_dataset, '{}.p'.format(CRSP_FLAG_FNAME), DATA, DROPBOX_ANALYSIS_DATA)

Serializing to local path /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/crsp_event_flags.p
Uploading /Users/lrraymond13/MIT/Kreiger_RA_2016/mkt_reaction_drug_failure/analysis/Data/crsp_event_flags.p to Dropbox as /data/intermediate/crsp_event_flags.p


In [45]:
crsp_flag_dataset.to_csv(os.path.join(DATA,'{}.csv'.format(CRSP_FLAG_FNAME)))

In [ ]:
crsp_flag_dataset.head()